In [100]:
from requests import get
from bs4 import BeautifulSoup
import requests
import json
from re import compile
import re
from IPython.display import JSON

In [101]:
class write_to_json:
    def __init__(self, data: dict, filename: str ='data.json'):
        self.data: dict = data
        self.filename: str = filename

    def __enter__(self) -> None:
        with open(self.filename, 'w') as f: 
            json.dump(self.data, f, indent=4, ensure_ascii=False)

    def __call__(self, *args, **kwargs) -> None:
        with open(self.filename, 'w') as f:
            json.dump(self.data, f, indent=4, ensure_ascii=False)

    @staticmethod
    def clear_data(data: str) -> str:
        reg: re.Pattern = compile("[\n]", re.VERBOSE)
        result: str = re.sub(" +", " ", reg.sub("", data))

        if len(result) == 0: return result
        if result[0] == " " and result[-1] == " ": return result[1:-1]
        elif result[0] == " ": return result[1:]
        elif result[-1] == " ": return result[:-1]
        else: return result

In [102]:
page: requests.models.Response = get("http://static.feed.rbc.ru/rbc/logical/footer/news.rss")

soup: BeautifulSoup = BeautifulSoup(page.content, 'html.parser')

links: list = soup.find_all('rbc_news:pdalink')


In [103]:
data: dict = {}

In [104]:
links = [link.text for link in links]

In [105]:
for link in links:
    page = get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    title = soup.find('h1', class_='article__header__title-in').text
    category = soup.find('a', class_='article__header__category').text
    time = soup.find('time', class_='article__header__date').text
    text = soup.find('div', class_='article__text article__text_free').text
    authors = soup.find_all('span', class_='article__authors__author__name')
    tags = soup.find_all('a', class_='article__tags__item')
    persons = soup.find_all('h3', class_='person-card__title')
    title = write_to_json.clear_data(title)
    text = write_to_json.clear_data(text)
    authors = [write_to_json.clear_data(author.text) for author in authors]
    tags = [write_to_json.clear_data(tag.text) for tag in tags]
    persons = [write_to_json.clear_data(person.text) for person in persons]
    if title != "" and text != "":
        data[title] = {
            "category": category,
            "time": time,
            "text": text,
            "link": link,
            "authors": authors,
            "tags": tags,
            "persons": persons
        }
        

In [106]:
#write_to_json(data).__enter__()
JSON(data)

<IPython.core.display.JSON object>